# **Bitcoin price prediction - Generalized Linear Regression (Walk Forward Split)**
### Big Data Computing final project - A.Y. 2022 - 2023
Prof. Gabriele Tolomei

MSc in Computer Science

La Sapienza, University of Rome

### Author: Corsi Danilo (1742375) - corsi.1742375@studenti.uniroma1.it


---


Description: perform model's train / validation with hyperparameter tuning and cross validation based on different methods of splitting the dataset.

# Global constants, dependencies, libraries and tools

In [1]:
# Main constants
LOCAL_RUNNING = True
SLOW_OPERATIONS = False # Decide whether or not to use operations that might slow down notebook execution
ROOT_DIR = "D:/Documents/Repository/BDC/project" if LOCAL_RUNNING else "/content/drive"

In [2]:
if not LOCAL_RUNNING:
    # Point Colaboratory to Google Drive
    from google.colab import drive

    # Define GDrive paths
    drive.mount(ROOT_DIR, force_remount=True)

    # Install Spark and related dependencies
    !pip install pyspark
    !pip install -U -q PyDrive -qq
    !apt install openjdk-8-jdk-headless -qq

## Import my utilities

In [3]:
# Set main dir
MAIN_DIR = ROOT_DIR + "" if LOCAL_RUNNING else ROOT_DIR + "/MyDrive/BDC/project"

# Utilities dir
UTILITIES_DIR = MAIN_DIR + "/utilities"

# Import my utilities
import sys
sys.path.append(UTILITIES_DIR)

from imports import *
import train_validation_utilities
from config import *

importlib.reload(train_validation_utilities)

<module 'train_validation_utilities' from 'D:\\Documents/Repository/BDC/project/utilities\\train_validation_utilities.py'>

## Core variables

In [4]:
# BS = Block Split
# WFS = Walk Forward Split
# SS = Single Split
SPLITTING_METHOD = WFS

# LR = LinearRegression 
# GLR = GeneralizedLinearRegression 
# RF = RandomForestRegressor 
# GBTR = GradientBoostingTreeRegressor
MODEL_NAME = GLR

In [5]:
###################
# --- DATASET --- #
###################

# Datasets dirs
DATASET_OUTPUT_DIR = MAIN_DIR + "/datasets/output"

# Datasets paths
DATASET_TRAIN_VALID  = DATASET_OUTPUT_DIR + "/" + DATASET_TRAIN_VALID_NAME + ".parquet"

####################
# --- FEATURES --- #
####################

# Features dir
FEATURES_DIR = MAIN_DIR + "/features"

# Features paths
FEATURES_CORRELATION = FEATURES_DIR + "/" + FEATURES_CORRELATION_LABEL + ".json"
BASE_FEATURES = FEATURES_DIR + "/" + BASE_FEATURES_LABEL + ".json"
BASE_AND_MOST_CORR_FEATURES = FEATURES_DIR + "/" + BASE_AND_MOST_CORR_FEATURES_LABEL + ".json"
BASE_AND_LEAST_CORR_FEATURES = FEATURES_DIR + "/" + BASE_AND_LEAST_CORR_FEATURES_LABEL + ".json"

##################
# --- MODELS --- #
##################

# Model dir
MODELS_DIR = MAIN_DIR + "/models"

# Model path
MODEL = MODELS_DIR + "/" + MODEL_NAME

###################
# --- RESULTS --- #
###################

# Results dir
RESULTS_DIR = MAIN_DIR + "/results/" + SPLITTING_METHOD

# Results path
ALL_MODEL_RESULTS  = RESULTS_DIR + "/" + MODEL_NAME + "_all.csv"
REL_MODEL_RESULTS  = RESULTS_DIR + "/" + MODEL_NAME + "_rel.csv"

MODEL_ACCURACY_RESULTS  = RESULTS_DIR + "/" + MODEL_NAME + "_accuracy.csv"

In [6]:
# Importing useful libraries
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

pio.renderers.default = 'vscode+colab' # To correctly render plotly plots

# Create the pyspark session

In [7]:
# Create the session
conf = SparkConf().\
                set('spark.ui.port', "4050").\
                set('spark.executor.memory', '12G').\
                set('spark.driver.memory', '12G').\
                set('spark.driver.maxResultSize', '109G').\
                set("spark.kryoserializer.buffer.max", "1G").\
                setAppName("BitcoinPricePrediction").\
                setMaster("local[*]")

# Create the context
sc = pyspark.SparkContext(conf=conf)
spark = SparkSession.builder.getOrCreate()

# Loading dataset

In [8]:
# Load train / validation set into pyspark dataset objects
df = spark.read.load(DATASET_TRAIN_VALID,
                         format="parquet",
                         sep=",",
                         inferSchema="true",
                         header="true"
                    )

In [9]:
train_validation_utilities.dataset_info(df)

+-------------------+---+-----------------+-----------------+-----------------+-----------------+-----------------+------------------+-----------------+--------------------+--------------------+------------------+------------------+--------------------+------------------------+-------------------+------------------+--------------------+--------------------+------------------+--------------+--------------------------------+-----------------+-----------------+-----------------+-----------------+-----------------+-----------------+-----------------+
|          timestamp| id|     market-price|    opening-price|    highest-price|     lowest-price|    closing-price|  trade-volume-btc|   total-bitcoins|          market-cap|    trade-volume-usd|       blocks-size|    avg-block-size|n-transactions-total|n-transactions-per-block|          hash-rate|        difficulty|      miners-revenue|transaction-fees-usd|n-unique-addresses|n-transactions|estimated-transaction-volume-usd|       sma-5-days|   

# Loading features

In [10]:
# Loading base features
with open(BASE_FEATURES, "r") as f:
    BASE_FEATURES = json.load(f)
print(BASE_FEATURES)

['opening-price', 'highest-price', 'lowest-price', 'closing-price', 'trade-volume-btc', 'market-price', 'market-cap', 'total-bitcoins', 'trade-volume-usd']


In [11]:
# Loading currency and additional most correlated features
with open(BASE_AND_MOST_CORR_FEATURES, "r") as f:
    BASE_AND_MOST_CORR_FEATURES = json.load(f)
print(BASE_AND_MOST_CORR_FEATURES)

['opening-price', 'highest-price', 'lowest-price', 'closing-price', 'trade-volume-btc', 'market-price', 'market-cap', 'total-bitcoins', 'trade-volume-usd', 'miners-revenue', 'sma-5-days', 'sma-7-days', 'sma-10-days', 'estimated-transaction-volume-usd', 'sma-20-days']


In [12]:
# Loading currency and additional least correlated features
with open(BASE_AND_LEAST_CORR_FEATURES, "r") as f:
    BASE_AND_LEAST_CORR_FEATURES = json.load(f)
print(BASE_AND_LEAST_CORR_FEATURES)

['opening-price', 'highest-price', 'lowest-price', 'closing-price', 'trade-volume-btc', 'market-price', 'market-cap', 'total-bitcoins', 'trade-volume-usd', 'sma-100-days', 'transaction-fees-usd', 'n-unique-addresses', 'sma-50-days', 'n-transactions-total', 'blocks-size', 'hash-rate', 'difficulty', 'avg-block-size', 'n-transactions-per-block', 'n-transactions']


# Model train / validation
In order to train and validate the model I'll try several approaches:
- `Default without normalization:` make predictions using the base model
- `Default with normalization:` like the previous one but features are normalized

Then the features that gave on average the most satisfactory results (for each model) are chosen and proceeded with:
- `Hyperparameter tuning:` finding the best parameters to use. 
- `Cross Validation:` validate the performance of the model with the chosen parameters (also here using Block split / Walk forward split)

If the final results are satisfactory, the model will be trained on the whole train / validation set and saved in order to make predictions on the test set.

For each approach the train / validation set will be split according to the chosen splitting method (in order to figure out which one works best for our problem). In this case the `Walk forward time series splits` method will be used: involves using a sliding window approach to create the training and validation sets for each fold. The model is trained on a fixed window of historical data, and then validated on the next observation in the time series. This process is repeated for each subsequent observation, with the window sliding forward one step at a time. 

<img src="https://github.com/CorsiDanilo/big-data-computing-project/blob/main/notebooks/images/walk-forward-splits.png?raw=1">

In [13]:
# Get splitting parameters based on the choosen splitting method
splitting_info = train_validation_utilities.get_splitting_params(SPLITTING_METHOD)
splitting_info

{'split_type': 'walk_forward_splits',
 'min_obser': 20000,
 'sliding_window': 5000}

## Default
The train / validation set will be splitted based on the splitting method chosen so that the model performance can be seen without any tuning by using different features (normalized and non)

In [14]:
# Get default parameters
params = train_validation_utilities.get_defaults_model_params(MODEL_NAME)
params

{'maxIter': [25], 'regParam': [0]}

### Without normalization

In [15]:
# Define model and features type
MODEL_TYPE = "default"
FEATURES_NORMALIZATION = False

In [16]:
# Choose base features
CHOSEN_FEATURES = BASE_FEATURES
CHOSEN_FEATURES_LABEL = BASE_FEATURES_LABEL

In [17]:
# Make predictions by using base features
default_train_results_base_features, default_valid_results_base_features, default_train_pred_base_features, default_valid_pred_base_features = train_validation_utilities.multiple_splits(df, params, splitting_info, MODEL_NAME, MODEL_TYPE, FEATURES_NORMALIZATION, CHOSEN_FEATURES, CHOSEN_FEATURES_LABEL, FEATURES_LABEL, TARGET_LABEL, SLOW_OPERATIONS)

Split: [22/22]: 100%|██████████| 1/1 [00:01<00:00,  1.28s/it]


In [18]:
default_train_results_base_features

,Model,Type,Dataset,Splitting,Features,Splits,Train / Validation,Parameters,RMSE,MSE,MAE,MAPE,R2,Adjusted_R2,Time
0,GeneralizedLinearRegression,default,train,walk_forward_splits,base_features,1,"(20000, 5000)","[25, 0]",2.435528,5.931795,1.775203,0.000222,0.999996,0.999996,1.137076
1,GeneralizedLinearRegression,default,train,walk_forward_splits,base_features,2,"(20000, 5000)","[25, 0]",2.475028,6.125762,1.772700,0.000211,0.999996,0.999996,0.331001
2,GeneralizedLinearRegression,default,train,walk_forward_splits,base_features,3,"(20000, 5000)","[25, 0]",2.501137,6.255685,1.802832,0.000205,0.999998,0.999998,0.313000
3,GeneralizedLinearRegression,default,train,walk_forward_splits,base_features,4,"(20000, 5000)","[25, 0]",2.453739,6.020836,1.733474,0.000163,0.999998,0.999998,0.290513
4,GeneralizedLinearRegression,default,train,walk_forward_splits,base_features,5,"(20000, 5000)","[25, 0]",3.709999,13.764096,2.449029,0.000166,0.999999,0.999999,0.254000
5,GeneralizedLinearRegression,default,train,walk_forward_splits,base_features,6,"(20000, 5000)","[25, 0]",9.638562,92.901869,5.483164,0.000218,0.999999,0.999999,0.263518
6,GeneralizedLinearRegression,default,train,walk_forward_splits,base_features,7,"(20000, 5000)","[25, 0]",12.194722,148.711252,7.983527,0.000236,1.000000,1.000000,0.255000
7,GeneralizedLinearRegression,default,train,walk_forward_splits,base_features,8,"(20000, 5000)","[25, 0]",15.897169,252.719991,11.076743,0.000272,0.999999,0.999999,0.258433
8,GeneralizedLinearRegression,default,train,walk_forward_splits,base_features,9,"(20000, 5000)","[25, 0]",16.404645,269.112368,11.976985,0.000277,0.999997,0.999997,0.258520
9,GeneralizedLinearRegression,default,train,walk_forward_splits,base_features,10,"(20000, 5000)","[25, 0]",15.268518,233.127637,11.170710,0.000249,0.999997,0.999997,0.258000


In [19]:
default_valid_results_base_features

,Model,Type,Dataset,Splitting,Features,Splits,Train / Validation,Parameters,RMSE,MSE,MAE,MAPE,R2,Adjusted_R2,Time
0,GeneralizedLinearRegression,default,valid,walk_forward_splits,base_features,1,"(20000, 5000)","[25, 0]",1.967400,3.870664,1.244890,0.000127,0.999990,0.999990,1.137076
1,GeneralizedLinearRegression,default,valid,walk_forward_splits,base_features,2,"(20000, 5000)","[25, 0]",2.605017,6.786111,1.925979,0.000172,0.999981,0.999981,0.331001
2,GeneralizedLinearRegression,default,valid,walk_forward_splits,base_features,3,"(20000, 5000)","[25, 0]",2.724320,7.421920,1.846456,0.000142,0.999998,0.999998,0.313000
3,GeneralizedLinearRegression,default,valid,walk_forward_splits,base_features,4,"(20000, 5000)","[25, 0]",6.471575,41.881283,5.010823,0.000230,0.999998,0.999998,0.290513
4,GeneralizedLinearRegression,default,valid,walk_forward_splits,base_features,5,"(20000, 5000)","[25, 0]",18.771401,352.365482,13.769709,0.000338,0.999994,0.999994,0.254000
5,GeneralizedLinearRegression,default,valid,walk_forward_splits,base_features,6,"(20000, 5000)","[25, 0]",16.145362,260.672702,12.597670,0.000228,0.999986,0.999986,0.263518
6,GeneralizedLinearRegression,default,valid,walk_forward_splits,base_features,7,"(20000, 5000)","[25, 0]",22.168812,491.456224,16.149503,0.000362,0.999994,0.999994,0.255000
7,GeneralizedLinearRegression,default,valid,walk_forward_splits,base_features,8,"(20000, 5000)","[25, 0]",11.347899,128.774810,9.291584,0.000266,0.999985,0.999985,0.258433
8,GeneralizedLinearRegression,default,valid,walk_forward_splits,base_features,9,"(20000, 5000)","[25, 0]",14.330402,205.360409,10.656071,0.000230,0.999976,0.999976,0.258520
9,GeneralizedLinearRegression,default,valid,walk_forward_splits,base_features,10,"(20000, 5000)","[25, 0]",24.800043,615.042153,13.522523,0.000233,0.999990,0.999990,0.258000


In [20]:
# Choose base and most additional correlated features
CHOSEN_FEATURES = BASE_AND_MOST_CORR_FEATURES
CHOSEN_FEATURES_LABEL = BASE_AND_MOST_CORR_FEATURES_LABEL

In [21]:
# Make predictions by using base and most additional correlated features
default_train_results_base_and_most_corr_features, default_valid_results_base_and_most_corr_features, default_train_pred_base_and_most_corr_features, default_valid_pred_base_and_most_corr_features = train_validation_utilities.multiple_splits(df, params, splitting_info, MODEL_NAME, MODEL_TYPE, FEATURES_NORMALIZATION, CHOSEN_FEATURES, CHOSEN_FEATURES_LABEL, FEATURES_LABEL, TARGET_LABEL, SLOW_OPERATIONS)

Split: [22/22]: 100%|██████████| 1/1 [00:01<00:00,  1.36s/it]


In [22]:
default_train_results_base_and_most_corr_features

,Model,Type,Dataset,Splitting,Features,Splits,Train / Validation,Parameters,RMSE,MSE,MAE,MAPE,R2,Adjusted_R2,Time
0,GeneralizedLinearRegression,default,train,walk_forward_splits,base_and_most_corr_features,1,"(20000, 5000)","[25, 0]",2.394387,5.733089,1.764614,0.000221,0.999996,0.999996,0.290000
1,GeneralizedLinearRegression,default,train,walk_forward_splits,base_and_most_corr_features,2,"(20000, 5000)","[25, 0]",2.429342,5.901701,1.761104,0.000209,0.999996,0.999996,0.381516
2,GeneralizedLinearRegression,default,train,walk_forward_splits,base_and_most_corr_features,3,"(20000, 5000)","[25, 0]",2.460705,6.055071,1.791034,0.000203,0.999998,0.999998,0.261001
3,GeneralizedLinearRegression,default,train,walk_forward_splits,base_and_most_corr_features,4,"(20000, 5000)","[25, 0]",2.424943,5.880348,1.736340,0.000164,0.999998,0.999998,0.254997
4,GeneralizedLinearRegression,default,train,walk_forward_splits,base_and_most_corr_features,5,"(20000, 5000)","[25, 0]",3.622242,13.120634,2.458340,0.000168,1.000000,1.000000,0.273249
5,GeneralizedLinearRegression,default,train,walk_forward_splits,base_and_most_corr_features,6,"(20000, 5000)","[25, 0]",9.281299,86.142503,5.468560,0.000224,0.999999,0.999999,0.278001
6,GeneralizedLinearRegression,default,train,walk_forward_splits,base_and_most_corr_features,7,"(20000, 5000)","[25, 0]",11.978819,143.492113,7.818047,0.000223,1.000000,1.000000,0.286518
7,GeneralizedLinearRegression,default,train,walk_forward_splits,base_and_most_corr_features,8,"(20000, 5000)","[25, 0]",15.637570,244.533607,11.058752,0.000271,0.999999,0.999999,0.295001
8,GeneralizedLinearRegression,default,train,walk_forward_splits,base_and_most_corr_features,9,"(20000, 5000)","[25, 0]",16.231205,263.452010,11.871252,0.000273,0.999997,0.999997,0.279000
9,GeneralizedLinearRegression,default,train,walk_forward_splits,base_and_most_corr_features,10,"(20000, 5000)","[25, 0]",15.202700,231.122095,11.136229,0.000248,0.999997,0.999997,0.292053


In [23]:
default_valid_results_base_and_most_corr_features

,Model,Type,Dataset,Splitting,Features,Splits,Train / Validation,Parameters,RMSE,MSE,MAE,MAPE,R2,Adjusted_R2,Time
0,GeneralizedLinearRegression,default,valid,walk_forward_splits,base_and_most_corr_features,1,"(20000, 5000)","[25, 0]",5.066869,25.673165,4.174077,0.000431,0.999934,0.999934,0.290000
1,GeneralizedLinearRegression,default,valid,walk_forward_splits,base_and_most_corr_features,2,"(20000, 5000)","[25, 0]",16.524886,273.071854,14.425821,0.001317,0.999232,0.999231,0.381516
2,GeneralizedLinearRegression,default,valid,walk_forward_splits,base_and_most_corr_features,3,"(20000, 5000)","[25, 0]",3.501878,12.263152,2.648493,0.000207,0.999996,0.999996,0.261001
3,GeneralizedLinearRegression,default,valid,walk_forward_splits,base_and_most_corr_features,4,"(20000, 5000)","[25, 0]",6.674011,44.542422,5.167971,0.000236,0.999998,0.999998,0.254997
4,GeneralizedLinearRegression,default,valid,walk_forward_splits,base_and_most_corr_features,5,"(20000, 5000)","[25, 0]",32.616080,1063.808683,26.151445,0.000633,0.999981,0.999981,0.273249
5,GeneralizedLinearRegression,default,valid,walk_forward_splits,base_and_most_corr_features,6,"(20000, 5000)","[25, 0]",36.323552,1319.400439,31.427627,0.000554,0.999930,0.999930,0.278001
6,GeneralizedLinearRegression,default,valid,walk_forward_splits,base_and_most_corr_features,7,"(20000, 5000)","[25, 0]",23.676308,560.567578,18.478149,0.000418,0.999993,0.999993,0.286518
7,GeneralizedLinearRegression,default,valid,walk_forward_splits,base_and_most_corr_features,8,"(20000, 5000)","[25, 0]",31.043815,963.718439,27.908732,0.000804,0.999887,0.999887,0.295001
8,GeneralizedLinearRegression,default,valid,walk_forward_splits,base_and_most_corr_features,9,"(20000, 5000)","[25, 0]",28.033908,785.899984,21.927642,0.000469,0.999908,0.999908,0.279000
9,GeneralizedLinearRegression,default,valid,walk_forward_splits,base_and_most_corr_features,10,"(20000, 5000)","[25, 0]",25.555276,653.072120,14.461111,0.000250,0.999989,0.999989,0.292053


In [24]:
# Choose base and least additional correlated features
CHOSEN_FEATURES = BASE_AND_LEAST_CORR_FEATURES
CHOSEN_FEATURES_LABEL = BASE_AND_LEAST_CORR_FEATURES_LABEL

In [25]:
# Make predictions by using base and least additional correlated features
default_train_results_base_and_least_corr_features, default_valid_results_base_and_least_corr_features, default_train_pred_base_and_least_corr_features, default_valid_pred_base_and_least_corr_features = train_validation_utilities.multiple_splits(df, params, splitting_info, MODEL_NAME, MODEL_TYPE, FEATURES_NORMALIZATION, CHOSEN_FEATURES, CHOSEN_FEATURES_LABEL, FEATURES_LABEL, TARGET_LABEL, SLOW_OPERATIONS)

Split: [22/22]: 100%|██████████| 1/1 [00:01<00:00,  1.57s/it]


In [26]:
default_train_results_base_and_least_corr_features

,Model,Type,Dataset,Splitting,Features,Splits,Train / Validation,Parameters,RMSE,MSE,MAE,MAPE,R2,Adjusted_R2,Time
0,GeneralizedLinearRegression,default,train,walk_forward_splits,base_and_least_corr_features,1,"(20000, 5000)","[25, 0]",2.404372,5.781006,1.771638,0.000222,0.999996,0.999996,0.378002
1,GeneralizedLinearRegression,default,train,walk_forward_splits,base_and_least_corr_features,2,"(20000, 5000)","[25, 0]",4.239158,17.970458,3.081568,0.000374,0.999989,0.999989,2.310926
2,GeneralizedLinearRegression,default,train,walk_forward_splits,base_and_least_corr_features,3,"(20000, 5000)","[25, 0]",4.511198,20.350910,3.265560,0.000380,0.999993,0.999993,0.488904
3,GeneralizedLinearRegression,default,train,walk_forward_splits,base_and_least_corr_features,4,"(20000, 5000)","[25, 0]",2.424511,5.878255,1.743085,0.000165,0.999998,0.999998,0.399994
4,GeneralizedLinearRegression,default,train,walk_forward_splits,base_and_least_corr_features,5,"(20000, 5000)","[25, 0]",7.986730,63.787854,5.541164,0.000383,0.999998,0.999998,0.424127
5,GeneralizedLinearRegression,default,train,walk_forward_splits,base_and_least_corr_features,6,"(20000, 5000)","[25, 0]",9.215036,84.916880,5.365513,0.000219,0.999999,0.999999,0.345834
6,GeneralizedLinearRegression,default,train,walk_forward_splits,base_and_least_corr_features,7,"(20000, 5000)","[25, 0]",33.097532,1095.446652,20.492264,0.000645,0.999996,0.999996,0.371924
7,GeneralizedLinearRegression,default,train,walk_forward_splits,base_and_least_corr_features,8,"(20000, 5000)","[25, 0]",36.868850,1359.312072,21.069773,0.000523,0.999993,0.999993,0.338073
8,GeneralizedLinearRegression,default,train,walk_forward_splits,base_and_least_corr_features,9,"(20000, 5000)","[25, 0]",36.653543,1343.482181,21.943948,0.000501,0.999987,0.999987,0.396000
9,GeneralizedLinearRegression,default,train,walk_forward_splits,base_and_least_corr_features,10,"(20000, 5000)","[25, 0]",29.266226,856.511964,20.348322,0.000454,0.999990,0.999990,0.373517


In [27]:
default_valid_results_base_and_least_corr_features

,Model,Type,Dataset,Splitting,Features,Splits,Train / Validation,Parameters,RMSE,MSE,MAE,MAPE,R2,Adjusted_R2,Time
0,GeneralizedLinearRegression,default,valid,walk_forward_splits,base_and_least_corr_features,1,"(20000, 5000)","[25, 0]",2.791788,7.794079,2.211872,0.000228,0.999980,0.999980,0.378002
1,GeneralizedLinearRegression,default,valid,walk_forward_splits,base_and_least_corr_features,2,"(20000, 5000)","[25, 0]",4.197800,17.621527,3.237724,0.000291,0.999950,0.999950,2.310926
2,GeneralizedLinearRegression,default,valid,walk_forward_splits,base_and_least_corr_features,3,"(20000, 5000)","[25, 0]",5.307284,28.167260,4.239820,0.000330,0.999991,0.999991,0.488904
3,GeneralizedLinearRegression,default,valid,walk_forward_splits,base_and_least_corr_features,4,"(20000, 5000)","[25, 0]",6.550041,42.903044,5.067042,0.000232,0.999998,0.999998,0.399994
4,GeneralizedLinearRegression,default,valid,walk_forward_splits,base_and_least_corr_features,5,"(20000, 5000)","[25, 0]",51.312083,2632.929909,31.370695,0.000774,0.999952,0.999952,0.424127
5,GeneralizedLinearRegression,default,valid,walk_forward_splits,base_and_least_corr_features,6,"(20000, 5000)","[25, 0]",21.708625,471.264410,17.842189,0.000323,0.999975,0.999975,0.345834
6,GeneralizedLinearRegression,default,valid,walk_forward_splits,base_and_least_corr_features,7,"(20000, 5000)","[25, 0]",81.513730,6644.488246,64.418107,0.001523,0.999915,0.999915,0.371924
7,GeneralizedLinearRegression,default,valid,walk_forward_splits,base_and_least_corr_features,8,"(20000, 5000)","[25, 0]",33.357450,1112.719503,27.193701,0.000792,0.999870,0.999869,0.338073
8,GeneralizedLinearRegression,default,valid,walk_forward_splits,base_and_least_corr_features,9,"(20000, 5000)","[25, 0]",48.830129,2384.381478,38.410888,0.000829,0.999720,0.999720,0.396000
9,GeneralizedLinearRegression,default,valid,walk_forward_splits,base_and_least_corr_features,10,"(20000, 5000)","[25, 0]",66.688928,4447.413141,34.237775,0.000587,0.999925,0.999925,0.373517


### With normalization

In [28]:
# Define model and features type
MODEL_TYPE = "default_norm"
FEATURES_NORMALIZATION = True

In [29]:
# Choose base features
CHOSEN_FEATURES = BASE_FEATURES
CHOSEN_FEATURES_LABEL = BASE_FEATURES_LABEL

In [30]:
# Make predictions by using base features
default_norm_train_results_base_features, default_norm_valid_results_base_features, default_norm_train_pred_base_features, default_norm_valid_pred_base_features = train_validation_utilities.multiple_splits(df, params, splitting_info, MODEL_NAME, MODEL_TYPE, FEATURES_NORMALIZATION, CHOSEN_FEATURES, CHOSEN_FEATURES_LABEL, FEATURES_LABEL, TARGET_LABEL, SLOW_OPERATIONS)

Split: [22/22]: 100%|██████████| 1/1 [00:01<00:00,  1.32s/it]


In [31]:
default_norm_train_results_base_features

,Model,Type,Dataset,Splitting,Features,Splits,Train / Validation,Parameters,RMSE,MSE,MAE,MAPE,R2,Adjusted_R2,Time
0,GeneralizedLinearRegression,default_norm,train,walk_forward_splits,base_features_norm,1,"(20000, 5000)","[25, 0]",172.449465,2.973882e+04,125.967961,0.016061,0.980736,0.980732,0.261000
1,GeneralizedLinearRegression,default_norm,train,walk_forward_splits,base_features_norm,2,"(20000, 5000)","[25, 0]",199.359256,3.974411e+04,137.223589,0.016516,0.975955,0.975950,0.216000
2,GeneralizedLinearRegression,default_norm,train,walk_forward_splits,base_features_norm,3,"(20000, 5000)","[25, 0]",339.923359,1.155479e+05,281.786687,0.031511,0.960132,0.960124,0.233001
3,GeneralizedLinearRegression,default_norm,train,walk_forward_splits,base_features_norm,4,"(20000, 5000)","[25, 0]",383.592607,1.471433e+05,243.133474,0.023506,0.952463,0.952454,0.237033
4,GeneralizedLinearRegression,default_norm,train,walk_forward_splits,base_features_norm,5,"(20000, 5000)","[25, 0]",1539.228214,2.369223e+06,1178.304354,0.086934,0.912996,0.912979,0.234998
5,GeneralizedLinearRegression,default_norm,train,walk_forward_splits,base_features_norm,6,"(20000, 5000)","[25, 0]",4300.165140,1.849142e+07,3350.967567,0.170925,0.882848,0.882825,0.242002
6,GeneralizedLinearRegression,default_norm,train,walk_forward_splits,base_features_norm,7,"(20000, 5000)","[25, 0]",5531.850165,3.060137e+07,4603.175821,0.201905,0.900913,0.900893,0.252513
7,GeneralizedLinearRegression,default_norm,train,walk_forward_splits,base_features_norm,8,"(20000, 5000)","[25, 0]",3601.173407,1.296845e+07,2975.002965,0.089996,0.935471,0.935458,0.249000
8,GeneralizedLinearRegression,default_norm,train,walk_forward_splits,base_features_norm,9,"(20000, 5000)","[25, 0]",1313.048981,1.724098e+06,1061.084232,0.024961,0.982816,0.982812,0.244001
9,GeneralizedLinearRegression,default_norm,train,walk_forward_splits,base_features_norm,10,"(20000, 5000)","[25, 0]",1400.508285,1.961423e+06,1163.720643,0.025762,0.976708,0.976703,0.254999


In [32]:
default_norm_valid_results_base_features

,Model,Type,Dataset,Splitting,Features,Splits,Train / Validation,Parameters,RMSE,MSE,MAE,MAPE,R2,Adjusted_R2,Time
0,GeneralizedLinearRegression,default_norm,valid,walk_forward_splits,base_features_norm,1,"(20000, 5000)","[25, 0]",264.307943,6.985869e+04,158.822298,0.015516,0.821762,0.821619,0.261000
1,GeneralizedLinearRegression,default_norm,valid,walk_forward_splits,base_features_norm,2,"(20000, 5000)","[25, 0]",797.245095,6.355997e+05,741.327145,0.064909,-0.787832,-0.789264,0.216000
2,GeneralizedLinearRegression,default_norm,valid,walk_forward_splits,base_features_norm,3,"(20000, 5000)","[25, 0]",1430.948083,2.047612e+06,1065.667243,0.076648,0.370242,0.369737,0.233001
3,GeneralizedLinearRegression,default_norm,valid,walk_forward_splits,base_features_norm,4,"(20000, 5000)","[25, 0]",6201.314710,3.845630e+07,5179.856522,0.222014,-0.919125,-0.920662,0.237033
4,GeneralizedLinearRegression,default_norm,valid,walk_forward_splits,base_features_norm,5,"(20000, 5000)","[25, 0]",15598.618764,2.433169e+08,13763.119202,0.319465,-3.452407,-3.455973,0.234998
5,GeneralizedLinearRegression,default_norm,valid,walk_forward_splits,base_features_norm,6,"(20000, 5000)","[25, 0]",15616.929439,2.438885e+08,15087.307380,0.265730,-11.902574,-11.912906,0.242002
6,GeneralizedLinearRegression,default_norm,valid,walk_forward_splits,base_features_norm,7,"(20000, 5000)","[25, 0]",5155.734711,2.658160e+07,4321.196705,0.096414,0.658871,0.658598,0.252513
7,GeneralizedLinearRegression,default_norm,valid,walk_forward_splits,base_features_norm,8,"(20000, 5000)","[25, 0]",7218.817557,5.211133e+07,6592.658222,0.193099,-5.107760,-5.112651,0.249000
8,GeneralizedLinearRegression,default_norm,valid,walk_forward_splits,base_features_norm,9,"(20000, 5000)","[25, 0]",2169.076227,4.704892e+06,2144.720314,0.046414,0.447885,0.447443,0.244001
9,GeneralizedLinearRegression,default_norm,valid,walk_forward_splits,base_features_norm,10,"(20000, 5000)","[25, 0]",3506.617620,1.229637e+07,3066.896252,0.051809,0.792092,0.791925,0.254999


In [33]:
# Choose base and most additional correlated features
CHOSEN_FEATURES = BASE_AND_MOST_CORR_FEATURES
CHOSEN_FEATURES_LABEL = BASE_AND_MOST_CORR_FEATURES_LABEL

In [34]:
# Make predictions by using base and most additional correlated features
default_norm_train_results_base_and_most_corr_features, default_norm_valid_results_base_and_most_corr_features, default_norm_train_pred_base_and_most_corr_features, default_norm_valid_pred_base_and_most_corr_features = train_validation_utilities.multiple_splits(df, params, splitting_info, MODEL_NAME, MODEL_TYPE, FEATURES_NORMALIZATION, CHOSEN_FEATURES, CHOSEN_FEATURES_LABEL, FEATURES_LABEL, TARGET_LABEL, SLOW_OPERATIONS)

Split: [22/22]: 100%|██████████| 1/1 [00:01<00:00,  1.53s/it]


In [35]:
default_norm_train_results_base_and_most_corr_features

,Model,Type,Dataset,Splitting,Features,Splits,Train / Validation,Parameters,RMSE,MSE,MAE,MAPE,R2,Adjusted_R2,Time
0,GeneralizedLinearRegression,default_norm,train,walk_forward_splits,base_and_most_corr_features_norm,1,"(20000, 5000)","[25, 0]",141.567250,2.004129e+04,104.019283,0.013299,0.987018,0.987015,0.317997
1,GeneralizedLinearRegression,default_norm,train,walk_forward_splits,base_and_most_corr_features_norm,2,"(20000, 5000)","[25, 0]",160.588821,2.578877e+04,119.642965,0.014321,0.984398,0.984395,0.274519
2,GeneralizedLinearRegression,default_norm,train,walk_forward_splits,base_and_most_corr_features_norm,3,"(20000, 5000)","[25, 0]",234.181954,5.484119e+04,185.870612,0.020492,0.981078,0.981074,0.271995
3,GeneralizedLinearRegression,default_norm,train,walk_forward_splits,base_and_most_corr_features_norm,4,"(20000, 5000)","[25, 0]",288.198946,8.305863e+04,198.890697,0.019033,0.973167,0.973161,0.271358
4,GeneralizedLinearRegression,default_norm,train,walk_forward_splits,base_and_most_corr_features_norm,5,"(20000, 5000)","[25, 0]",767.339216,5.888095e+05,578.971967,0.042743,0.978378,0.978373,0.306999
5,GeneralizedLinearRegression,default_norm,train,walk_forward_splits,base_and_most_corr_features_norm,6,"(20000, 5000)","[25, 0]",3072.492074,9.440208e+06,2459.702885,0.131297,0.940192,0.940180,0.328516
6,GeneralizedLinearRegression,default_norm,train,walk_forward_splits,base_and_most_corr_features_norm,7,"(20000, 5000)","[25, 0]",3081.815639,9.497588e+06,2564.857466,0.105676,0.969247,0.969241,0.342003
7,GeneralizedLinearRegression,default_norm,train,walk_forward_splits,base_and_most_corr_features_norm,8,"(20000, 5000)","[25, 0]",2706.646835,7.325937e+06,2250.175187,0.066399,0.963547,0.963540,0.327239
8,GeneralizedLinearRegression,default_norm,train,walk_forward_splits,base_and_most_corr_features_norm,9,"(20000, 5000)","[25, 0]",1303.031170,1.697890e+06,1079.839392,0.025299,0.983077,0.983074,0.292000
9,GeneralizedLinearRegression,default_norm,train,walk_forward_splits,base_and_most_corr_features_norm,10,"(20000, 5000)","[25, 0]",1094.476554,1.197879e+06,847.603451,0.019704,0.985775,0.985772,0.358001


In [36]:
default_norm_valid_results_base_and_most_corr_features

,Model,Type,Dataset,Splitting,Features,Splits,Train / Validation,Parameters,RMSE,MSE,MAE,MAPE,R2,Adjusted_R2,Time
0,GeneralizedLinearRegression,default_norm,valid,walk_forward_splits,base_and_most_corr_features_norm,1,"(20000, 5000)","[25, 0]",367.699700,1.352031e+05,348.666542,0.037065,0.655041,0.654765,0.317997
1,GeneralizedLinearRegression,default_norm,valid,walk_forward_splits,base_and_most_corr_features_norm,2,"(20000, 5000)","[25, 0]",648.688850,4.207972e+05,587.932183,0.051294,-0.183630,-0.184577,0.274519
2,GeneralizedLinearRegression,default_norm,valid,walk_forward_splits,base_and_most_corr_features_norm,3,"(20000, 5000)","[25, 0]",1310.927911,1.718532e+06,903.251923,0.063510,0.471453,0.471029,0.271995
3,GeneralizedLinearRegression,default_norm,valid,walk_forward_splits,base_and_most_corr_features_norm,4,"(20000, 5000)","[25, 0]",6680.537409,4.462958e+07,5471.637005,0.232418,-1.227196,-1.228980,0.271358
4,GeneralizedLinearRegression,default_norm,valid,walk_forward_splits,base_and_most_corr_features_norm,5,"(20000, 5000)","[25, 0]",12991.859489,1.687884e+08,11033.910516,0.251353,-2.088625,-2.091099,0.306999
5,GeneralizedLinearRegression,default_norm,valid,walk_forward_splits,base_and_most_corr_features_norm,6,"(20000, 5000)","[25, 0]",8229.028267,6.771691e+07,7530.807177,0.131103,-2.582467,-2.585336,0.328516
6,GeneralizedLinearRegression,default_norm,valid,walk_forward_splits,base_and_most_corr_features_norm,7,"(20000, 5000)","[25, 0]",35447.012833,1.256491e+09,27481.852300,0.695214,-15.124880,-15.137793,0.342003
7,GeneralizedLinearRegression,default_norm,valid,walk_forward_splits,base_and_most_corr_features_norm,8,"(20000, 5000)","[25, 0]",35118.036412,1.233276e+09,30034.089234,0.870320,-143.547392,-143.663146,0.327239
8,GeneralizedLinearRegression,default_norm,valid,walk_forward_splits,base_and_most_corr_features_norm,9,"(20000, 5000)","[25, 0]",1852.292055,3.430986e+06,1721.438745,0.037360,0.597377,0.597054,0.292000
9,GeneralizedLinearRegression,default_norm,valid,walk_forward_splits,base_and_most_corr_features_norm,10,"(20000, 5000)","[25, 0]",1914.271168,3.664434e+06,1311.146254,0.022683,0.938041,0.937992,0.358001


In [37]:
# Choose base and least additional correlated features
CHOSEN_FEATURES = BASE_AND_LEAST_CORR_FEATURES
CHOSEN_FEATURES_LABEL = BASE_AND_LEAST_CORR_FEATURES_LABEL

In [38]:
# Make predictions by using base and least additional correlated features
default_norm_train_results_base_and_least_corr_features, default_norm_valid_results_base_and_least_corr_features, default_norm_train_pred_base_and_least_corr_features, default_norm_valid_pred_base_and_least_corr_features = train_validation_utilities.multiple_splits(df, params, splitting_info, MODEL_NAME, MODEL_TYPE, FEATURES_NORMALIZATION, CHOSEN_FEATURES, CHOSEN_FEATURES_LABEL, FEATURES_LABEL, TARGET_LABEL, SLOW_OPERATIONS)

Split: [22/22]: 100%|██████████| 1/1 [00:01<00:00,  1.56s/it]


In [39]:
default_norm_train_results_base_and_least_corr_features

,Model,Type,Dataset,Splitting,Features,Splits,Train / Validation,Parameters,RMSE,MSE,MAE,MAPE,R2,Adjusted_R2,Time
0,GeneralizedLinearRegression,default_norm,train,walk_forward_splits,base_and_least_corr_features_norm,1,"(20000, 5000)","[25, 0]",46.775858,2187.980904,36.789900,0.004593,0.998583,0.998582,0.346002
1,GeneralizedLinearRegression,default_norm,train,walk_forward_splits,base_and_least_corr_features_norm,2,"(20000, 5000)","[25, 0]",58.359949,3405.883590,44.785131,0.005310,0.997939,0.997939,0.315054
2,GeneralizedLinearRegression,default_norm,train,walk_forward_splits,base_and_least_corr_features_norm,3,"(20000, 5000)","[25, 0]",85.924301,7382.985433,68.274485,0.007807,0.997453,0.997452,0.351999
3,GeneralizedLinearRegression,default_norm,train,walk_forward_splits,base_and_least_corr_features_norm,4,"(20000, 5000)","[25, 0]",79.020024,6244.164114,59.178084,0.005548,0.997983,0.997982,0.332514
4,GeneralizedLinearRegression,default_norm,train,walk_forward_splits,base_and_least_corr_features_norm,5,"(20000, 5000)","[25, 0]",101.742433,10351.522705,79.533996,0.006228,0.999620,0.999620,0.363000
5,GeneralizedLinearRegression,default_norm,train,walk_forward_splits,base_and_least_corr_features_norm,6,"(20000, 5000)","[25, 0]",265.644271,70566.878601,199.327453,0.010041,0.999553,0.999553,0.342517
6,GeneralizedLinearRegression,default_norm,train,walk_forward_splits,base_and_least_corr_features_norm,7,"(20000, 5000)","[25, 0]",423.833075,179634.475354,345.303284,0.012438,0.999418,0.999418,0.362998
7,GeneralizedLinearRegression,default_norm,train,walk_forward_splits,base_and_least_corr_features_norm,8,"(20000, 5000)","[25, 0]",464.127898,215414.705768,377.009728,0.010959,0.998928,0.998928,0.402524
8,GeneralizedLinearRegression,default_norm,train,walk_forward_splits,base_and_least_corr_features_norm,9,"(20000, 5000)","[25, 0]",686.845447,471756.668342,518.619391,0.012263,0.995298,0.995297,0.392002
9,GeneralizedLinearRegression,default_norm,train,walk_forward_splits,base_and_least_corr_features_norm,10,"(20000, 5000)","[25, 0]",734.188296,539032.453459,581.739895,0.013189,0.993599,0.993598,0.373999


In [40]:
default_norm_valid_results_base_and_least_corr_features

,Model,Type,Dataset,Splitting,Features,Splits,Train / Validation,Parameters,RMSE,MSE,MAE,MAPE,R2,Adjusted_R2,Time
0,GeneralizedLinearRegression,default_norm,valid,walk_forward_splits,base_and_least_corr_features_norm,1,"(20000, 5000)","[25, 0]",151.429406,2.293087e+04,104.902314,0.010443,0.941494,0.941447,0.346002
1,GeneralizedLinearRegression,default_norm,valid,walk_forward_splits,base_and_least_corr_features_norm,2,"(20000, 5000)","[25, 0]",264.081218,6.973889e+04,234.853585,0.020507,0.803837,0.803680,0.315054
2,GeneralizedLinearRegression,default_norm,valid,walk_forward_splits,base_and_least_corr_features_norm,3,"(20000, 5000)","[25, 0]",449.883773,2.023954e+05,372.570671,0.028047,0.937752,0.937702,0.351999
3,GeneralizedLinearRegression,default_norm,valid,walk_forward_splits,base_and_least_corr_features_norm,4,"(20000, 5000)","[25, 0]",1554.349606,2.416003e+06,1144.631145,0.046924,0.879432,0.879335,0.332514
4,GeneralizedLinearRegression,default_norm,valid,walk_forward_splits,base_and_least_corr_features_norm,5,"(20000, 5000)","[25, 0]",2550.349753,6.504284e+06,2113.359180,0.047900,0.880979,0.880884,0.363000
5,GeneralizedLinearRegression,default_norm,valid,walk_forward_splits,base_and_least_corr_features_norm,6,"(20000, 5000)","[25, 0]",1127.199169,1.270578e+06,908.863665,0.016300,0.932782,0.932728,0.342517
6,GeneralizedLinearRegression,default_norm,valid,walk_forward_splits,base_and_least_corr_features_norm,7,"(20000, 5000)","[25, 0]",5392.702134,2.908124e+07,4546.555216,0.106833,0.626793,0.626494,0.362998
7,GeneralizedLinearRegression,default_norm,valid,walk_forward_splits,base_and_least_corr_features_norm,8,"(20000, 5000)","[25, 0]",4834.891534,2.337618e+07,4097.027458,0.121293,-1.739828,-1.742022,0.402524
8,GeneralizedLinearRegression,default_norm,valid,walk_forward_splits,base_and_least_corr_features_norm,9,"(20000, 5000)","[25, 0]",2212.921675,4.897022e+06,2128.964333,0.045467,0.425339,0.424878,0.392002
9,GeneralizedLinearRegression,default_norm,valid,walk_forward_splits,base_and_least_corr_features_norm,10,"(20000, 5000)","[25, 0]",3532.458849,1.247827e+07,2994.222933,0.048972,0.789016,0.788847,0.373999


In [41]:
# Define model information and evaluators to show
model_info = ['Model', 'Type', 'Dataset', 'Splitting', 'Features', 'Parameters']
evaluator_lst = ['RMSE', 'MSE', 'MAE', 'MAPE', 'R2', 'Adjusted_R2', 'Time']

In [42]:
# Define the results to show
default_comparison_lst = [default_valid_results_base_features, default_valid_results_base_and_most_corr_features, default_valid_results_base_and_least_corr_features, default_norm_valid_results_base_features, default_norm_valid_results_base_and_most_corr_features, default_norm_valid_results_base_and_least_corr_features]

# Show the comparison table
default_comparison_lst_df = pd.concat([train_validation_utilities.model_comparison(results, model_info, evaluator_lst) for results in default_comparison_lst])
default_comparison_lst_df

,Model,Type,Dataset,Splitting,Features,Parameters,RMSE,MSE,MAE,MAPE,R2,Adjusted_R2,Time
0,GeneralizedLinearRegression,default,valid,walk_forward_splits,base_features,"[25, 0]",10.149807,1.442297e+02,7.225374,0.000220,0.999979,0.999979,0.306197
0,GeneralizedLinearRegression,default,valid,walk_forward_splits,base_and_most_corr_features,"[25, 0]",15.620374,3.414772e+02,12.467853,0.000411,0.999893,0.999892,0.296510
0,GeneralizedLinearRegression,default,valid,walk_forward_splits,base_and_least_corr_features,"[25, 0]",21.541681,9.162115e+02,15.600488,0.000450,0.999938,0.999938,0.467246
0,GeneralizedLinearRegression,default_norm,valid,walk_forward_splits,base_features_norm,"[25, 0]",4375.915428,3.971494e+07,3734.129210,0.117251,-8.120562,-8.127866,0.256122
0,GeneralizedLinearRegression,default_norm,valid,walk_forward_splits,base_and_most_corr_features_norm,"[25, 0]",6662.119726,1.350926e+08,5598.776134,0.174056,-17.573728,-17.588602,0.319716
0,GeneralizedLinearRegression,default_norm,valid,walk_forward_splits,base_and_least_corr_features_norm,"[25, 0]",1743.625346,5.340021e+06,1496.804362,0.043785,0.360428,0.359915,0.375512


In [43]:
# Save the best default model results and predicitons
best_default_results = pd.concat([train_validation_utilities.model_comparison(results, model_info, evaluator_lst) for results in [default_norm_valid_results_base_and_least_corr_features]])
best_default_predictions = default_norm_valid_pred_base_and_least_corr_features
best_default_results

,Model,Type,Dataset,Splitting,Features,Parameters,RMSE,MSE,MAE,MAPE,R2,Adjusted_R2,Time
0,GeneralizedLinearRegression,default_norm,valid,walk_forward_splits,base_and_least_corr_features_norm,"[25, 0]",1743.625346,5.340021e+06,1496.804362,0.043785,0.360428,0.359915,0.375512


In [45]:
# Define the choosen features
FEATURES_NORMALIZATION = True
CHOSEN_FEATURES = BASE_AND_LEAST_CORR_FEATURES
CHOSEN_FEATURES_LABEL = BASE_AND_LEAST_CORR_FEATURES_LABEL

## Tuned
Once the features and execution method are selected, the model will undergo hyperparameter tuning and cross validation to find the best configuration. 

In [46]:
# Get model grid parameters
params = train_validation_utilities.get_model_grid_params(MODEL_NAME)
params

{'maxIter': [5, 10, 50, 80],
 'regParam': [0, 0.1, 0.2],
 'family': ['gaussian', 'gamma'],
 'link': ['log', 'identity', 'inverse']}

### Hyperparameter tuning

In [47]:
# Select the type of feature to be used
MODEL_TYPE = "hyp_tuning"

In [48]:
# Perform hyperparameter tuning
hyp_res = train_validation_utilities.multiple_splits(df, params, splitting_info, MODEL_NAME, MODEL_TYPE, FEATURES_NORMALIZATION, CHOSEN_FEATURES, CHOSEN_FEATURES_LABEL, FEATURES_LABEL, TARGET_LABEL, SLOW_OPERATIONS)
hyp_res

Split: [1/22]: 100%|██████████| 72/72 [01:08<00:00,  1.05it/s]


Best parameters chosen for split [1/22]: [5, 0, 'gaussian', 'log']


Split: [2/22]: 100%|██████████| 72/72 [01:05<00:00,  1.10it/s]


Best parameters chosen for split [2/22]: [5, 0, 'gaussian', 'log']


Split: [3/22]: 100%|██████████| 72/72 [01:04<00:00,  1.11it/s]


...

Split: [21/22]: 100%|██████████| 72/72 [01:09<00:00,  1.03it/s]


Best parameters chosen for split [21/22]: [5, 0.1, 'gaussian', 'log']


Split: [22/22]: 100%|██████████| 72/72 [01:10<00:00,  1.03it/s]

Best parameters chosen for split [22/22]: [5, 0.2, 'gaussian', 'log']


,Model,Type,Dataset,Splitting,Features,Splits,Train / Validation,Parameters,RMSE,MSE,MAE,MAPE,R2,Adjusted_R2,Time
0,GeneralizedLinearRegression,hyp_tuning,valid,walk_forward_splits,base_and_least_corr_features_norm,1,"(20000, 5000)","[5, 0, gaussian, log]",151.429406,2.293087e+04,104.902314,0.010443,0.941494,0.941447,0.325999
1,GeneralizedLinearRegression,hyp_tuning,valid,walk_forward_splits,base_and_least_corr_features_norm,2,"(20000, 5000)","[5, 0, gaussian, log]",264.081218,6.973889e+04,234.853585,0.020507,0.803837,0.803680,0.311999
2,GeneralizedLinearRegression,hyp_tuning,valid,walk_forward_splits,base_and_least_corr_features_norm,3,"(20000, 5000)","[5, 0, gaussian, log]",449.883773,2.023954e+05,372.570671,0.028047,0.937752,0.937702,0.281000
3,GeneralizedLinearRegression,hyp_tuning,valid,walk_forward_splits,base_and_least_corr_features_norm,4,"(20000, 5000)","[5, 0.2, gaussian, log]",1529.426182,2.339144e+06,1139.095351,0.046882,0.883267,0.883174,0.075002
4,GeneralizedLinearRegression,hyp_tuning,valid,walk_forward_splits,base_and_least_corr_features_norm,5,"(20000, 5000)","[5, 0.2, gaussian, log]",2548.080473,6.492714e+06,2129.331311,0.048433,0.881191,0.881096,0.068000
5,GeneralizedLinearRegression,hyp_tuning,valid,walk_forward_splits,base_and_least_corr_features_norm,6,"(20000, 5000)","[5, 0.1, gaussian, log]",1081.289249,1.169186e+06,885.882067,0.015895,0.938146,0.938096,0.075001
6,GeneralizedLinearRegression,hyp_tuning,valid,walk_forward_splits,base_and_least_corr_features_norm,7,"(20000, 5000)","[5, 0.2, gaussian, log]",4963.379465,2.463514e+07,4184.796259,0.098078,0.683851,0.683598,0.080999
7,GeneralizedLinearRegression,hyp_tuning,valid,walk_forward_splits,base_and_least_corr_features_norm,8,"(20000, 5000)","[5, 0.2, gaussian, log]",4688.180657,2.197904e+07,3928.473897,0.116173,-1.576075,-1.578138,0.073002
8,GeneralizedLinearRegression,hyp_tuning,valid,walk_forward_splits,base_and_least_corr_features_norm,9,"(20000, 5000)","[5, 0.1, gaussian, log]",1661.222624,2.759661e+06,1523.308590,0.032495,0.676156,0.675897,0.072001
9,GeneralizedLinearRegression,hyp_tuning,valid,walk_forward_splits,base_and_least_corr_features_norm,10,"(20000, 5000)","[5, 0, gaussian, log]",3532.458849,1.247827e+07,2994.222933,0.048972,0.789016,0.788847,0.288000


---
Since during this stage will be used the Block split / Walk forward split method of the dataset I compute a score for each parameter chosen by each split, assigning weights based on:
   * Their `frequency` for each split (if the same parameters are chosen from several splits, these will have greater weight) 
   * The `split` they belong to (the closer the split is to today's date the more weight they will have)
   * Their `RMSE value` for each split (the lower this is, the more weight they will have)
   
   Then, the overall score will be calculated by putting together these three weights for each parameter and the one with the best score will be the chosen parameter.

In [49]:
# Show parameters score
grouped_scores, best_params = train_validation_utilities.choose_best_params(hyp_res)
grouped_scores

,Split weight,RMSE weight,Frequency weight,Final score
Parameters,,,,
"(5, 0.1, gaussian, log)",0.643939,0.849635,0.75,0.410335
"(5, 0, gaussian, log)",0.471591,0.821476,1.00,0.387401
"(5, 0.2, gaussian, log)",0.482955,0.493070,1.00,0.238130


In [50]:
# Print best parameters
print(f"Best parameters: {best_params}")

Best parameters: (5, 0.1, 'gaussian', 'log')


### Cross validation

In [51]:
MODEL_TYPE = "cross_val"

In [52]:
# Get tuned parameters
params = train_validation_utilities.get_best_model_params(best_params, MODEL_NAME)
params

{'maxIter': [5], 'regParam': [0.1], 'family': ['gaussian'], 'link': ['log']}

In [53]:
# Perform cross validation
cv_train_result, cv_valid_result, cv_train_pred, cv_valid_pred = train_validation_utilities.multiple_splits(df, params, splitting_info, MODEL_NAME, MODEL_TYPE, FEATURES_NORMALIZATION, CHOSEN_FEATURES, CHOSEN_FEATURES_LABEL, FEATURES_LABEL, TARGET_LABEL, SLOW_OPERATIONS)

Split: [22/22]: 100%|██████████| 1/1 [00:01<00:00,  1.20s/it]


In [54]:
cv_train_result

,Model,Type,Dataset,Splitting,Features,Splits,Train / Validation,Parameters,RMSE,MSE,MAE,MAPE,R2,Adjusted_R2,Time
0,GeneralizedLinearRegression,cross_val,train,walk_forward_splits,base_and_least_corr_features_norm,1,"(20000, 5000)","[5, 0.1, gaussian, log]",45.480116,2068.440952,36.072439,0.004485,0.998660,0.998660,0.225003
1,GeneralizedLinearRegression,cross_val,train,walk_forward_splits,base_and_least_corr_features_norm,2,"(20000, 5000)","[5, 0.1, gaussian, log]",58.688092,3444.292120,45.043282,0.005340,0.997916,0.997916,0.299000
2,GeneralizedLinearRegression,cross_val,train,walk_forward_splits,base_and_least_corr_features_norm,3,"(20000, 5000)","[5, 0.1, gaussian, log]",85.220886,7262.599439,67.952751,0.007769,0.997494,0.997494,0.267998
3,GeneralizedLinearRegression,cross_val,train,walk_forward_splits,base_and_least_corr_features_norm,4,"(20000, 5000)","[5, 0.1, gaussian, log]",78.222414,6118.746113,58.551019,0.005493,0.998023,0.998023,0.267514
4,GeneralizedLinearRegression,cross_val,train,walk_forward_splits,base_and_least_corr_features_norm,5,"(20000, 5000)","[5, 0.1, gaussian, log]",98.453337,9693.059480,77.243504,0.006063,0.999644,0.999644,0.200000
5,GeneralizedLinearRegression,cross_val,train,walk_forward_splits,base_and_least_corr_features_norm,6,"(20000, 5000)","[5, 0.1, gaussian, log]",263.723210,69549.931504,197.490969,0.009892,0.999559,0.999559,0.261996
6,GeneralizedLinearRegression,cross_val,train,walk_forward_splits,base_and_least_corr_features_norm,7,"(20000, 5000)","[5, 0.1, gaussian, log]",426.124197,181581.831120,344.125634,0.012171,0.999412,0.999412,0.270002
7,GeneralizedLinearRegression,cross_val,train,walk_forward_splits,base_and_least_corr_features_norm,8,"(20000, 5000)","[5, 0.1, gaussian, log]",450.966166,203370.482477,368.334409,0.010756,0.998988,0.998988,0.260526
8,GeneralizedLinearRegression,cross_val,train,walk_forward_splits,base_and_least_corr_features_norm,9,"(20000, 5000)","[5, 0.1, gaussian, log]",676.162588,457195.845532,509.593742,0.012039,0.995443,0.995442,0.264997
9,GeneralizedLinearRegression,cross_val,train,walk_forward_splits,base_and_least_corr_features_norm,10,"(20000, 5000)","[5, 0.1, gaussian, log]",689.809502,475837.148737,542.496837,0.012364,0.994349,0.994348,0.230999


In [55]:
cv_valid_result

,Model,Type,Dataset,Splitting,Features,Splits,Train / Validation,Parameters,RMSE,MSE,MAE,MAPE,R2,Adjusted_R2,Time
0,GeneralizedLinearRegression,cross_val,valid,walk_forward_splits,base_and_least_corr_features_norm,1,"(20000, 5000)","[5, 0.1, gaussian, log]",174.110686,3.031453e+04,134.742080,0.013601,0.922655,0.922593,0.225003
1,GeneralizedLinearRegression,cross_val,valid,walk_forward_splits,base_and_least_corr_features_norm,2,"(20000, 5000)","[5, 0.1, gaussian, log]",283.896812,8.059740e+04,260.330206,0.022836,0.773293,0.773112,0.299000
2,GeneralizedLinearRegression,cross_val,valid,walk_forward_splits,base_and_least_corr_features_norm,3,"(20000, 5000)","[5, 0.1, gaussian, log]",455.691031,2.076543e+05,378.885004,0.028552,0.936134,0.936083,0.267998
3,GeneralizedLinearRegression,cross_val,valid,walk_forward_splits,base_and_least_corr_features_norm,4,"(20000, 5000)","[5, 0.1, gaussian, log]",1531.521915,2.345559e+06,1136.415340,0.046714,0.882947,0.882853,0.267514
4,GeneralizedLinearRegression,cross_val,valid,walk_forward_splits,base_and_least_corr_features_norm,5,"(20000, 5000)","[5, 0.1, gaussian, log]",2559.933715,6.553261e+06,2143.771978,0.048802,0.880083,0.879987,0.200000
5,GeneralizedLinearRegression,cross_val,valid,walk_forward_splits,base_and_least_corr_features_norm,6,"(20000, 5000)","[5, 0.1, gaussian, log]",1081.289249,1.169186e+06,885.882067,0.015895,0.938146,0.938096,0.261996
6,GeneralizedLinearRegression,cross_val,valid,walk_forward_splits,base_and_least_corr_features_norm,7,"(20000, 5000)","[5, 0.1, gaussian, log]",5061.089369,2.561463e+07,4266.581115,0.100057,0.671281,0.671017,0.270002
7,GeneralizedLinearRegression,cross_val,valid,walk_forward_splits,base_and_least_corr_features_norm,8,"(20000, 5000)","[5, 0.1, gaussian, log]",4747.003649,2.253404e+07,3974.361524,0.117501,-1.641125,-1.643240,0.260526
8,GeneralizedLinearRegression,cross_val,valid,walk_forward_splits,base_and_least_corr_features_norm,9,"(20000, 5000)","[5, 0.1, gaussian, log]",1661.222624,2.759661e+06,1523.308590,0.032495,0.676156,0.675897,0.264997
9,GeneralizedLinearRegression,cross_val,valid,walk_forward_splits,base_and_least_corr_features_norm,10,"(20000, 5000)","[5, 0.1, gaussian, log]",3979.312474,1.583493e+07,3461.274115,0.057276,0.732261,0.732047,0.230999


In [56]:
# Define the results to show
tuned_comparison_lst = [cv_valid_result]

# Show the comparison table
tuned_comparison_lst_df = pd.concat([train_validation_utilities.model_comparison(results, model_info, evaluator_lst) for results in tuned_comparison_lst])
tuned_comparison_lst_df

,Model,Type,Dataset,Splitting,Features,Parameters,RMSE,MSE,MAE,MAPE,R2,Adjusted_R2,Time
0,GeneralizedLinearRegression,cross_val,valid,walk_forward_splits,base_and_least_corr_features_norm,"[5, 0.1, gaussian, log]",1525.049433,4.367730e+06,1293.502172,0.038137,0.60912,0.608807,0.247618


# Comparison table
Visualization of model performance at various stages of train / validation

In [57]:
# Concatenate final results into Pandas dataset
final_comparison_lst_df = pd.DataFrame(pd.concat([default_comparison_lst_df, tuned_comparison_lst_df], ignore_index=True))
final_comparison_lst_df

,Model,Type,Dataset,Splitting,Features,Parameters,RMSE,MSE,MAE,MAPE,R2,Adjusted_R2,Time
0,GeneralizedLinearRegression,default,valid,walk_forward_splits,base_features,"[25, 0]",10.149807,1.442297e+02,7.225374,0.000220,0.999979,0.999979,0.306197
1,GeneralizedLinearRegression,default,valid,walk_forward_splits,base_and_most_corr_features,"[25, 0]",15.620374,3.414772e+02,12.467853,0.000411,0.999893,0.999892,0.296510
2,GeneralizedLinearRegression,default,valid,walk_forward_splits,base_and_least_corr_features,"[25, 0]",21.541681,9.162115e+02,15.600488,0.000450,0.999938,0.999938,0.467246
3,GeneralizedLinearRegression,default_norm,valid,walk_forward_splits,base_features_norm,"[25, 0]",4375.915428,3.971494e+07,3734.129210,0.117251,-8.120562,-8.127866,0.256122
4,GeneralizedLinearRegression,default_norm,valid,walk_forward_splits,base_and_most_corr_features_norm,"[25, 0]",6662.119726,1.350926e+08,5598.776134,0.174056,-17.573728,-17.588602,0.319716
5,GeneralizedLinearRegression,default_norm,valid,walk_forward_splits,base_and_least_corr_features_norm,"[25, 0]",1743.625346,5.340021e+06,1496.804362,0.043785,0.360428,0.359915,0.375512
6,GeneralizedLinearRegression,cross_val,valid,walk_forward_splits,base_and_least_corr_features_norm,"[5, 0.1, gaussian, log]",1525.049433,4.367730e+06,1293.502172,0.038137,0.609120,0.608807,0.247618


# Model accuracy

Since predicting the price accurately is very difficult I also saw how good the models are at predicting whether the price will go up or down in this way:

For each prediction let's consider the actual market-price, next-market-price and our predicted next-market-price (prediction).
I compute whether the current prediction is correct (1) or not (0):

$$ 
prediction\_is\_correct
= 
\begin{cases}
0 \text{ if [(market-price > next-market-price) and (market-price < prediction)] or [(market-price < next-market-price) and (market-price > prediction)]} \\
1 \text{ if [(market-price > next-market-price) and (market-price > prediction)] or [(market-price < next-market-price) and (market-price < prediction)]}
\end{cases}
$$

After that I count the number of correct prediction:
$$ 
correct\_predictions
= 
\sum_{i=0}^{total\_rows} prediction\_is\_correct
$$

Finally I compute the percentage of accuracy of the model:
$$
\\ 
accuracy 
= 
(correct\_predictions / total\_rows) 
* 100
$$

In [58]:
# Convert the pandas dataset to a PySpark dataset
best_default_pred_spark = spark.createDataFrame(best_default_predictions)
validated_pred_spark = spark.createDataFrame(cv_valid_pred)

# Compute model accuracy
default_accuracy = train_validation_utilities.model_accuracy(best_default_pred_spark)
validated_accuracy = train_validation_utilities.model_accuracy(validated_pred_spark)

# Shows whether features are normalised or not
if FEATURES_NORMALIZATION:
    NEW_CHOSEN_FEATURES_LABEL = CHOSEN_FEATURES_LABEL + "_norm"
    CHOSEN_FEATURES_LABEL = NEW_CHOSEN_FEATURES_LABEL
    
# Saving accuracy data into dataframe
accuracy_data = {
    'Model': MODEL_NAME,
    'Features': CHOSEN_FEATURES_LABEL,
    'Splitting': SPLITTING_METHOD,
    'Accuracy (default)': default_accuracy,
    'Accuracy (tuned)': validated_accuracy
}
accuracy_data_df = pd.DataFrame(accuracy_data, index=['Model'])

print(f"Percentage of correct predictions for {MODEL_NAME} with {CHOSEN_FEATURES_LABEL} and {SPLITTING_METHOD} (default): {default_accuracy:.2f}%")
print(f"Percentage of correct predictions for {MODEL_NAME} with {CHOSEN_FEATURES_LABEL} and {SPLITTING_METHOD} (tuned): {validated_accuracy:.2f}%")

Percentage of correct predictions for GeneralizedLinearRegression with base_and_least_corr_features_norm and walk_forward_splits (default): 48.02%
Percentage of correct predictions for GeneralizedLinearRegression with base_and_least_corr_features_norm and walk_forward_splits (tuned): 47.99%


In [59]:
# Concatenate default and tuned results
default_tuned_results = [best_default_results, cv_valid_result]
default_tuned_results_df = pd.concat([train_validation_utilities.model_comparison(results, model_info, evaluator_lst) for results in default_tuned_results])
default_tuned_results_df

,Model,Type,Dataset,Splitting,Features,Parameters,RMSE,MSE,MAE,MAPE,R2,Adjusted_R2,Time
0,GeneralizedLinearRegression,default_norm,valid,walk_forward_splits,base_and_least_corr_features_norm,"[25, 0]",1743.625346,5.340021e+06,1496.804362,0.043785,0.360428,0.359915,0.375512
0,GeneralizedLinearRegression,cross_val,valid,walk_forward_splits,base_and_least_corr_features_norm,"[5, 0.1, gaussian, log]",1525.049433,4.367730e+06,1293.502172,0.038137,0.609120,0.608807,0.247618


# Saving final results

In [60]:
# Save all final comparison results
final_comparison_lst_df.to_csv(ALL_MODEL_RESULTS, index=False)

In [61]:
# Save relevant results (default and tuned results)
default_tuned_results_df.to_csv(REL_MODEL_RESULTS, index=False)

In [62]:
# Saving accuracy results
accuracy_data_df.to_csv(MODEL_ACCURACY_RESULTS, index=False)

In [63]:
# Export notebook in html format (remember to save the notebook and change the model name)
if LOCAL_RUNNING:
    !jupyter nbconvert --to html 4-walk-forward-split_{MODEL_NAME}.ipynb --output 4-walk-forward-split_{MODEL_NAME} --output-dir='./exports'

d:\Documents\Repository\BDC\project\.bdc\Lib\site-packages\traitlets\traitlets.py:2548: FutureWarning: Supporting extra quotes around strings is deprecated in traitlets 5.0. You can use './exports' instead of "'./exports'" if you require traitlets >=5.
  warn(
[NbConvertApp] Converting notebook 4-walk-forward-split_GeneralizedLinearRegression.ipynb to html
[NbConvertApp] WARNING | Alternative text is missing on 1 image(s).
[NbConvertApp] Writing 543496 bytes to exports\4-walk-forward-split_GeneralizedLinearRegression.html
